In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,confusion_matrix,auc
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer,precision_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

## Read Files

In [2]:
review_train = pd.read_csv(r"E:\Data science\DataHack\Skillenza\Stage 1 Level 1\training.csv")
review_test = pd.read_csv(r"E:\Data science\DataHack\Skillenza\Stage 1 Level 1\test_.csv")

In [3]:
submission = pd.read_csv(r"E:\Data science\DataHack\Skillenza\Stage 1 Level 1\sample_submission_.csv")

## Train

In [4]:
review_train.head()

,id,category,text
0,959,0,5573 1189 4017 1207 4768 8542 17 1189 5085 5773
1,994,0,6315 7507 6700 4742 1944 2692 3647 4413 6700
2,995,0,5015 8067 5335 1615 7957 5773
3,996,0,2925 7199 1994 4647 7455 5773 4518 2734 2807 8...
4,997,0,7136 1207 6781 237 4971 3669 6193


In [5]:
review_train.shape

(3464, 3)

In [6]:
review_train.category.value_counts()

0    3348
1     116
Name: category, dtype: int64

## Test

In [7]:
review_test.head()

,id,text
0,3729,2705 4888 5050 5815 2472 5157 652 2117 2110 32...
1,3732,389 4978 315 5178 513 5249 5853 3267 315 3891 ...
2,3761,4478 4231 4858 2638 4231 867 371 686 4888 4179...
3,5,3015 1911 112 3905 825 337 315 1693 4677 825 5...
4,7,5136 3918 5153 2023 3091 4159 315 3711 1409 27...


In [405]:
review_test.shape

(1360, 2)

## Check Train-Test Overlap

In [9]:
print(len(list(set(review_train.text)&set(review_test.text))))
print(len(list(set(review_train.id)&set(review_test.id))))

0
0


## Bag of words (One hot Encoding)

Since Text is already in numeric form so no need of Data Preprocessing such as lower case, stemming, Stopwords removal etc

In [10]:
# For train
countvector=CountVectorizer()
td_train = countvector.fit_transform(review_train.text)

# print(pd.DataFrame(td.A, columns=countvector.get_feature_names()).to_string())
df_train = pd.DataFrame(td_train.toarray(), columns=countvector.get_feature_names())


# For test
td_test = countvector.transform(review_test.text)

df_test = pd.DataFrame(td_test.toarray(), columns=countvector.get_feature_names())

In [11]:
df_train.shape , df_test.shape

((3464, 8484), (1360, 8484))

## tf-idf

In [409]:
# For train
tf = TfidfVectorizer()
vector_train = tf.fit_transform(review_train.text).toarray()
df_train = pd.DataFrame(vector_train, columns=tf.get_feature_names())

# For test
# tf = TfidfVectorizer()
vector_test = tf.transform(review_test.text).toarray()
df_test = pd.DataFrame(vector_test, columns=tf.get_feature_names())

## Handling Class Imbalance

In [386]:
def class_weight(labels_dict,trainy,mu = 0.15):
    total = len(trainy)
    keys = labels_dict.keys()
    weight = dict()
    for i in keys:
        score = np.log(mu * (total)/float(labels_dict[i]))
        weight[i] = score if score > 1 else 1
    return weight

In [15]:
def ClassBalance(trainy , minor_class_weight_coeff):
    classes = np.unique(trainy, axis = 0) # classes in y variable 
    classes.sort()
    class_value_count = trainy.value_counts().values # value count of each class(it returns only numbers)
    n_observations = len(trainy) # total no. of rows
    n_classes = trainy.nunique() # n. of classes
    weights = list(n_observations / (n_classes *class_value_count*1.0))
    weights[1] = weights[1] * minor_class_weight_coeff
    sample_weights = np.where(trainy.isin([classes[0]]),weights[0],weights[1])
    return sample_weights

## Model Building

In [16]:
# Stratified Kfold
# Suppose, the dataset has 90 observations of negative class and 10 observations of positive class, then ideal value of 
# scale_pos_weight should be 9.
# Learning Rate 0.2 0.7909
import imblearn
from imblearn.over_sampling import SMOTE


Kfolds = StratifiedKFold(n_splits=10,shuffle = True , random_state = 101)
i = 1
accuracy = []
f1_score_class1 = []
index_train = []
index_val = []
predicted =  []
pred_TEST = []

# XGBOOST
for train_index,val_index in Kfolds.split(df_train,review_train['category']):
    train_x , val_x = df_train.iloc[train_index], df_train.iloc[val_index]
    train_y , val_y = review_train['category'].iloc[train_index], review_train['category'].iloc[val_index]
    
    smt = SMOTE(sampling_strategy=0.7)
    td_tr, y_train = smt.fit_sample(train_x, train_y)
    
    index_train.append([train_index])
    index_val.append([val_index])
    #weights = ClassBalance(train_y , 0.4)
    #labels_dict = train_y.value_counts().to_dict()
    #weights = class_weight()
    
    from xgboost import XGBClassifier
    xgc = XGBClassifier(learning_rate = 0.2,n_jobs = -1)
    xgc.fit(train_x , train_y)#, sample_weight = weights)
    valid_pred = xgc.predict(val_x) ; train_pred = xgc.predict(train_x)
    valid_score = accuracy_score(valid_pred,val_y) ; train_score = accuracy_score(train_pred, train_y)
    predicted.append(valid_pred)
    
    
    print("For CV = ",i)
    print(confusion_matrix(valid_pred,val_y))
    print("\nClassification Report\n",classification_report(valid_pred,val_y))
    print("\nF1_Score:",f1_score(valid_pred,val_y)) # For negative class
    print("\nAccuracy for Validation:",valid_score,'||',"\tAccuracy for Train:",train_score)
    print("\n***************")
    i = i+1    
    accuracy.append(valid_score) ; f1_score_class1.append(f1_score(valid_pred,val_y))
    
    pred_TEST.append(list(xgc.predict(df_test)))
    
print("Mean Accuracy",np.mean(accuracy),"\nMean F1_Score:",np.mean(f1_score_class1))

For CV =  1
[[333   3]
 [  2   9]]

Classification Report
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       336
           1       0.75      0.82      0.78        11

    accuracy                           0.99       347
   macro avg       0.87      0.90      0.89       347
weighted avg       0.99      0.99      0.99       347


F1_Score: 0.7826086956521738

Accuracy for Validation: 0.9855907780979827 || 	Accuracy for Train: 0.9964709656721207

***************
For CV =  2
[[335   3]
 [  0   9]]

Classification Report
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       338
           1       0.75      1.00      0.86         9

    accuracy                           0.99       347
   macro avg       0.88      1.00      0.93       347
weighted avg       0.99      0.99      0.99       347


F1_Score: 0.8571428571428571

Accuracy for Validation: 0.9913544668587896 || 	Accuracy for T

## Hyperparameter Tuning

In [410]:
# Train Set
ndx_train = np.concatenate((np.array(index_val[0]).reshape(347,),np.array(index_val[1]).reshape(347,),
                            np.array(index_val[2]).reshape(347,),np.array(index_val[3]).reshape(347,),
                            np.array(index_val[7]).reshape(346,),np.array(index_val[8]).reshape(346,),
                            np.array(index_val[9]).reshape(346,),))
print(ndx_train.shape)
train_set_x = df_train.iloc[ndx_train,:]
train_set_y = review_train.category.loc[ndx_train]


# Development Set
ndx_dev = np.concatenate((np.array(index_val[4]).reshape(346,),
                         np.array(index_val[5]).reshape(346,),
                         np.array(index_val[6]).reshape(346,)))
print(ndx_dev.shape)
dev_set_x = df_train.iloc[ndx_dev,:]
dev_set_y = review_train.category.loc[ndx_dev]

(2426,)
(1038,)


In [100]:
#### SMOTE
smt = SMOTE(sampling_strategy=0.8)
train_set_x, train_set_y = smt.fit_sample(train_set_x, train_set_y)

In [56]:
# XGBoost
xb = XGBClassifier(learning_rate = 0.3,max_depth=10)
xb.fit(train_set_x , train_set_y)
y_pred = xb.predict(dev_set_x) ; y_train_pred = xb.predict(train_set_x)
val_score = accuracy_score(y_pred,dev_set_y) ; tr_score = accuracy_score(y_train_pred, train_set_y)
    
    
print(confusion_matrix(y_pred,dev_set_y))
print("\nClassification Report\n",classification_report(y_pred,dev_set_y))
print("\nF1_Score:",f1_score(y_pred,dev_set_y)) # For negative class
print("\nAccuracy for Validation:",val_score,'||',"\tAccuracy for Train:",tr_score)
print("\n***************")

[[1003   14]
 [   2   19]]

Classification Report
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      1017
           1       0.58      0.90      0.70        21

    accuracy                           0.98      1038
   macro avg       0.79      0.95      0.85      1038
weighted avg       0.99      0.98      0.99      1038


F1_Score: 0.7037037037037038

Accuracy for Validation: 0.9845857418111753 || 	Accuracy for Train: 0.998351195383347

***************


In [414]:
# LGBM (lr = 0.24 , depth = 12 , 17,19 , )
lgb = LGBMClassifier(learning_rate = 0.03, class_weight = weights)
lgb.fit(train_set_x , train_set_y)
y_pred = lgb.predict(dev_set_x) ; y_train_pred = lgb.predict(train_set_x)
val_score = accuracy_score(y_pred,dev_set_y) ; tr_score = accuracy_score(y_train_pred, train_set_y)


print(confusion_matrix(y_pred,dev_set_y))
print("\nClassification Report\n",classification_report(y_pred,dev_set_y))
print("\nF1_Score:",f1_score(y_pred,dev_set_y)) # For negative class
print("\nAccuracy for Validation:",val_score,'||',"\tAccuracy for Train:",tr_score)
print("\n***************")

[[1005   12]
 [   0   21]]

Classification Report
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      1017
           1       0.64      1.00      0.78        21

    accuracy                           0.99      1038
   macro avg       0.82      0.99      0.89      1038
weighted avg       0.99      0.99      0.99      1038


F1_Score: 0.7777777777777778

Accuracy for Validation: 0.9884393063583815 || 	Accuracy for Train: 1.0

***************


In [415]:
pd.Series(lgb.predict(df_test)).value_counts()

0    1360
dtype: int64

In [84]:
train_set.shape , dev_set.shape

((2426, 5653), (1038, 5653))

In [118]:
# Number of trees in xgboost
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 3)]
# Various learning rate parameters
learning_rate = ['0.02','0.1', '0.2','0.3']# ,'0.3','0.5','0.6','0.7','0.9']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
#Subssample parameter values: fraction of the training samples (randomly selected) that will be used to train each tree.
subsample=[0.7,0.6,0.8]
# Minimum child weight parameters: Similar to min_split in RF
# min_child_weight=[3,4,5,6,7]
# colsample_bytree: fraction of features (randomly selected) that will be used to train each tree.
colsample_bytree = [0.5,0.7,0.8]
# colsample_bylevel: fraction of features (randomly selected) that will be used to train each tree.
# colsample_bylevel = [0.5,0.7,0.8]


# Create the grid
random_grid = {# 'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'subsample': subsample,
               # 'min_child_weight': min_child_weight,
               'colsample_bytree': colsample_bytree}
#               'colsample_bylevel': colsample_bylevel}

In [ ]:
xgboost = XGBClassifier()
xg_tune = GridSearchCV(xgboost, random_grid, scoring = 'accuracy', cv = 9, verbose = 2) # here i am doing 7 fold cross validation
xg_tune.fit(train_set_x , train_set_y)

In [119]:
lg = LGBMClassifier()
lg_tune = GridSearchCV(lg, random_grid, scoring = 'accuracy', cv = 9, verbose = 2) # here i am doing 7 fold cross validation
lg_tune.fit(train_set_x , train_set_y)

Fitting 9 folds for each of 216 candidates, totalling 1944 fits
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.9s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.7s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.6s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.8s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.6s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.6s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7, total=   1.6s
[CV] colsample_bytree=0.5, learning_rate=0.02, max_depth=5, subsample=0.7 
[CV]  cols

...

[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   1.9s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   1.9s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   2.2s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8, total=   2.5s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=5, subsample=0.8 
[CV]  colsample_bytree

[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.6, total=   2.1s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.6, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.6, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8, total=   2.0s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8, total=   2.8s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=15, subsample=0.8 
[CV]  cols

[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   2.6s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   2.6s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   3.2s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   2.8s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   2.6s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6, total=   2.3s
[CV] colsample_bytree=0.8, learning_rate=0.3, max_depth=25, subsample=0.6 
[CV]  cols

[Parallel(n_jobs=1)]: Done 1944 out of 1944 | elapsed: 65.5min finished


GridSearchCV(cv=9, estimator=LGBMClassifier(),
             param_grid={'colsample_bytree': [0.5, 0.7, 0.8],
                         'learning_rate': ['0.02', '0.1', '0.2', '0.3'],
                         'max_depth': [5, 10, 15, 20, 25, 30],
                         'subsample': [0.7, 0.6, 0.8]},
             scoring='accuracy', verbose=2)

In [120]:
lg_tune.best_estimator_

LGBMClassifier(colsample_bytree=0.8, learning_rate='0.3', max_depth=5,
               subsample=0.7)

## Kfold on LGBM

In [121]:
# Learning rate = 0.02 , sampling_strategy = 0.8 --> F1_score 0.84419 , Accuracy: 99.076

#### SMOTE
import imblearn
from imblearn.over_sampling import SMOTE



# Stratified Kfold

Kfolds = StratifiedKFold(n_splits=10,shuffle = True , random_state = 0)
i = 1
accuracy = []
f1_score_class1 = []
index_train = []
index_val = []
predicted =  []
pred_test1 = []


# LGBM
for train_index,val_index in Kfolds.split(df_train,review_train['category']):
    train_x , val_x = df_train.iloc[train_index], df_train.iloc[val_index]
    train_y , val_y = review_train['category'].iloc[train_index], review_train['category'].iloc[val_index]
#     smt = SMOTE(sampling_strategy=0.7)
#     td_tr, y_train = smt.fit_sample(train_x, train_y)
    
    
    index_train.append([train_index])
    index_val.append([val_index])
    #weights = ClassBalance(train_y , 0.3)
    
    from lightgbm import LGBMClassifier
    lgb = LGBMClassifier(colsample_bytree=0.8, learning_rate='0.3', max_depth=5,
               subsample=0.7)
    lgb.fit(train_x , train_y)#, sample_weight = weights)
    valid_pred = lgb.predict(val_x) ; train_pred = lgb.predict(train_x)
    valid_score = accuracy_score(valid_pred,val_y) ; train_score = accuracy_score(train_pred, train_y)
    predicted.append(valid_pred)
    
    
    print("For CV = ",i)
    print(confusion_matrix(valid_pred,val_y))
    print("\nClassification Report\n",classification_report(valid_pred,val_y))
    print("\nF1_Score:",f1_score(valid_pred,val_y)) # For negative class
    print("\nAccuracy for Validation:",valid_score,'||',"\tAccuracy for Train:",train_score)
    print("\n***************")
    i = i+1    
    accuracy.append(valid_score) ; f1_score_class1.append(f1_score(valid_pred,val_y))
    
    pred_test1.append(list(lgb.predict(df_test)))
    
print("Mean Accuracy",np.mean(accuracy),"\nMean F1_Score:",np.mean(f1_score_class1))

For CV =  1
[[333   2]
 [  2  10]]

Classification Report
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       335
           1       0.83      0.83      0.83        12

    accuracy                           0.99       347
   macro avg       0.91      0.91      0.91       347
weighted avg       0.99      0.99      0.99       347


F1_Score: 0.8333333333333334

Accuracy for Validation: 0.9884726224783862 || 	Accuracy for Train: 1.0

***************
For CV =  2
[[335   2]
 [  0  10]]

Classification Report
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       337
           1       0.83      1.00      0.91        10

    accuracy                           0.99       347
   macro avg       0.92      1.00      0.95       347
weighted avg       1.00      0.99      0.99       347


F1_Score: 0.9090909090909091

Accuracy for Validation: 0.9942363112391931 || 	Accuracy for Train: 1.0

****

In [385]:
df_test[overlap].shape

(1360, 5653)

In [116]:
# implement Naive Bayes

import numpy as np
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,confusion_matrix


Kfolds = StratifiedKFold(n_splits=10,shuffle = True , random_state = 0)
i = 1
accuracy = []
f1_score_class1 = []
index_train1 = []
index_val1 = []
predicted1 =  []
pred_test = []

for train_index,val_index in Kfolds.split(df_train,review_train['category']):
    train_x , val_x = df_train.iloc[train_index], df_train.iloc[val_index]
    train_y , val_y = review_train['category'].iloc[train_index], review_train['category'].iloc[val_index]
    index_train1.append([train_index])
    index_val1.append([val_index])
    weights1 = ClassBalance(train_y , 0.59)
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(train_x , train_y, sample_weight = weights1)
    valid_pred = nb.predict(val_x) ; train_pred = nb.predict(train_x)
    valid_score = accuracy_score(valid_pred,val_y) ; train_score = accuracy_score(train_pred, train_y)
    predicted1.append(valid_pred)
    print("For CV = ",i)
    print(confusion_matrix(valid_pred,val_y))
    print("\nF1_Score:",f1_score(valid_pred,val_y)) # For negative class
    print("\nAccuracy for Validation:",valid_score,'||',"\tAccuracy for Train:",train_score)    
#    print("\nClassification Report\n",classification_report(y_pred,val_y))
    print("\n***************")
    i = i+1
    accuracy.append(valid_score) ; f1_score_class1.append(f1_score(valid_pred,val_y))
    #y = nb.predict(df_test[overlap])
    pred_test.append(list(nb.predict(df_test)))
    
print("Mean Accuracy",np.mean(accuracy),"\nMean F1_Score:",np.mean(f1_score_class1))

For CV =  1
[[334   0]
 [  1  12]]

F1_Score: 0.9600000000000001

Accuracy for Validation: 0.9971181556195965 || 	Accuracy for Train: 0.9980750721847931

***************
For CV =  2
[[333   1]
 [  2  11]]

F1_Score: 0.8799999999999999

Accuracy for Validation: 0.9913544668587896 || 	Accuracy for Train: 0.9980750721847931

***************
For CV =  3
[[331   1]
 [  4  11]]

F1_Score: 0.8148148148148148

Accuracy for Validation: 0.9855907780979827 || 	Accuracy for Train: 0.9980750721847931

***************
For CV =  4
[[334   3]
 [  1   9]]

F1_Score: 0.8181818181818182

Accuracy for Validation: 0.9884726224783862 || 	Accuracy for Train: 0.9980750721847931

***************
For CV =  5
[[333   1]
 [  2  10]]

F1_Score: 0.8695652173913043

Accuracy for Validation: 0.9913294797687862 || 	Accuracy for Train: 0.9983964079538166

***************
For CV =  6
[[333   0]
 [  2  11]]

F1_Score: 0.9166666666666666

Accuracy for Validation: 0.9942196531791907 || 	Accuracy for Train: 0.99775497113534

## Neural Net

In [184]:
import tensorflow as tf
import keras

In [190]:
tr = pd.read_csv(r"E:\Data science\DataHack\Skillenza\Stage 1 Level 1\training.csv")
tst = pd.read_csv(r"E:\Data science\DataHack\Skillenza\Stage 1 Level 1\test_.csv")

In [191]:
# Train test split
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(tr['text'] , tr['category'] , test_size = 0.3 , random_state = 0 , 
                                                    shuffle = True)

# Or take train and dev set formed while hyperparameter tuning

In [192]:
x_train.shape , x_val.shape, y_train.shape , y_val.shape

((2424,), (1040,), (2424,), (1040,))

In [288]:
# Count_Vectorizer

countvector = CountVectorizer()
td_tr = countvector.fit_transform(x_train).toarray()
td_val = countvector.transform(x_val).toarray()


# Full test
full_test = countvector.transform(review_test.text).toarray()

In [289]:
y_train = np.array(y_train)
y_val = np.array(y_val)

#### SMOTE

In [156]:
import imblearn
from imblearn.over_sampling import SMOTE

In [157]:
pd.Series(y_val).value_counts()

0    1014
1      26
dtype: int64

In [158]:
smt = SMOTE(sampling_strategy=0.7)
td_tr, y_train = smt.fit_sample(td_tr, y_train)

In [159]:
td_tr.shape , y_train.shape

((4201, 7170), (4201,))

In [160]:
# weight = ClassBalance(pd.Series(y_train) , 0.4)

In [290]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
td_tr = scl.fit_transform(td_tr)
td_val = scl.transform(td_val)

In [291]:
score = {'accuracy':make_scorer(accuracy_score), 'precision':make_scorer(precision_score), 'f1_score':make_scorer(f1_score)}

In [292]:
from keras.callbacks import EarlyStopping

In [355]:
# Building Neural Net;  400,400,2,lr = 0.01

model = tf.keras.models.Sequential() # initialize the network
model.add(tf.keras.layers.Dense(100 , activation = tf.nn.relu, input_dim = 7129)) # 1st hidden layer with 128 neurons and activation is relu
model.add(tf.keras.layers.Dense(100 , activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(300 , activation = tf.nn.relu))
# model.add(tf.keras.layers.Dense(800 , activation = tf.nn.tanh)) # 2nd Hidden layer with 128 neurons and activation is relu
# model.add(tf.keras.layers.Dense(1000 , activation = tf.nn.relu)) # 3rd hidden layer with 128 neurons and activation is relu
model.add(tf.keras.layers.Dense(2 , activation = tf.nn.softmax)) # output layer use the soft max instead of relu
# Dense means we are building fully connected layers.
# In O/p layer no. of neurons = class levels in y variables.
adam = tf.keras.optimizers.Adam(lr= 0.01)

model.compile( optimizer = adam, loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [356]:
td_tr.shape , td_val.shape , y_train.shape,y_val.shape

((2424, 7129), (1040, 7129), (2424,), (1040,))

In [357]:
model.fit(td_tr, y_train, batch_size = 512, epochs = 20, validation_data = (td_val , y_val),callbacks = [earlystop])

Train on 2424 samples, validate on 1040 samples
Epoch 1/20
2424/2424 [==============================] - 1s 521us/sample - loss: 0.3955 - acc: 0.9109 - val_loss: 0.1635 - val_acc: 0.9798
Epoch 2/20
2424/2424 [==============================] - 0s 180us/sample - loss: 0.2163 - acc: 0.9872 - val_loss: 0.1789 - val_acc: 0.9875
Epoch 3/20
2424/2424 [==============================] - 0s 184us/sample - loss: 0.0316 - acc: 0.9967 - val_loss: 0.1110 - val_acc: 0.9913
Epoch 4/20
2424/2424 [==============================] - 0s 181us/sample - loss: 0.0017 - acc: 0.9992 - val_loss: 0.0814 - val_acc: 0.9952
Epoch 5/20
2424/2424 [==============================] - 0s 179us/sample - loss: 0.0026 - acc: 0.9983 - val_loss: 0.0817 - val_acc: 0.9962
Epoch 6/20
2424/2424 [==============================] - 0s 181us/sample - loss: 6.9578e-04 - acc: 0.9992 - val_loss: 0.0818 - val_acc: 0.9962
Epoch 7/20
2424/2424 [==============================] - 0s 182us/sample - loss: 6.6637e-04 - acc: 0.9992 - val_loss: 0.0

In [358]:
pred = model.predict_classes(td_val)

In [359]:
print(confusion_matrix(pred,y_val))
print("\nF1_Score:",f1_score(pred,y_val)) # For negative class
print("\nAccuracy for Validation:",accuracy_score(pred,y_val),'||',"\tAccuracy for Train:",accuracy_score(pred,y_val))    
print("\nClassification Report\n",classification_report(pred,y_val))

[[1001    3]
 [   2   34]]

F1_Score: 0.9315068493150684

Accuracy for Validation: 0.9951923076923077 || 	Accuracy for Train: 0.9951923076923077

Classification Report
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1004
           1       0.92      0.94      0.93        36

    accuracy                           1.00      1040
   macro avg       0.96      0.97      0.96      1040
weighted avg       1.00      1.00      1.00      1040



In [360]:
pd.Series(model.predict_classes(full_test)).value_counts()

0    1352
1       8
dtype: int64

In [418]:
# Applying decision tree
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(review_train['text'], review_train["category"], test_size = .2,random_state = 101)

# For train
countvector=CountVectorizer()
td_train1 = countvector.fit_transform(x_train).toarray()
td_test1 = countvector.transform(x_test).toarray()
# print(pd.DataFrame(td.A, columns=countvector.get_feature_names()).to_string())
#df_train = pd.DataFrame(td_train.toarray(), columns=countvector.get_feature_names())


# For test
test_vector = countvector.transform(review_test['text']).toarray()

#df_test = pd.DataFrame(td_test.toarray(), columns=countvector.get_feature_names())

In [419]:
td_train1.shape , td_test1.shape

((2771, 7593), (693, 7593))

In [423]:
dtree = DecisionTreeClassifier(criterion= "entropy")


print(dtree.fit(td_train1, y_train))
dt_pred = dtree.predict(td_test1)

tab = confusion_matrix(dt_pred, y_test)
print(tab)

print(classification_report(dt_pred , y_test))

DecisionTreeClassifier(criterion='entropy')
[[678   5]
 [  3   7]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       683
           1       0.58      0.70      0.64        10

    accuracy                           0.99       693
   macro avg       0.79      0.85      0.82       693
weighted avg       0.99      0.99      0.99       693



## Submission

In [421]:
submission.drop(columns = 'category',inplace = True)
submission['category'] = pd.Series(dtree.predict(test_vector)).values

In [401]:
submission.drop(columns = 'category',inplace = True)
submission['category'] = pd.DataFrame(pred_test1).transpose().mode(axis = 1)[0]

In [361]:
submission.drop(columns = 'category',inplace = True)
submission['category'] = pd.Series(model.predict_classes(full_test)).values

In [402]:
submission.head()

,id,category
0,3729,0.0
1,3732,0.0
2,3761,0.0
3,5,0.0
4,7,0.0


In [422]:
submission.category.value_counts()

0    1335
1      25
Name: category, dtype: int64

In [404]:
excel = pd.ExcelWriter('Movie_review_DT1.xlsx')
submission.to_excel(excel)
excel.save()

DataFrame is written successfully to Excel File.
